In [1]:
import ruamel.yaml
import os
import re

In [ ]:
dbt_base_path = ''
if not dbt_base_path:
    dbt_base_path = get_destination()

In [2]:
dbt_base_path = "/Users/tim/Desktop/arc-mercycorps-dbt"

In [3]:
yaml = ruamel.yaml.YAML()
yaml.indent(mapping=4, sequence=4, offset=2)
yaml.preserve_quotes = True

In [4]:
with open('../../arc-standard-dashboard-dbt-fake-data-generator/.github/workflows/dbt_run_on_merge.yml','r') as f:
    content = f.read()
    workflow_yml = yaml.load(content)

In [5]:
starting_prompt = """You'll need to do some setup to have this util run.

First, you'll need to have a working dbt Github Repo.

Second, you'll need to have a dbtCloud project set up.
If you've never done this before, it's worth talking to an engineer.
Here are the docs if you'd like to try on your own:
https://docs.getdbt.com/docs/get-started/getting-started/set-up-dbt-cloud

Third, you need to set up an environment in dbt Cloud. Here are docs:
https://docs.getdbt.com/docs/get-started/getting-started/building-your-first-project/schedule-a-job#create-a-deployment-environment

Fourth, you need to create a job in dbt Cloud. Here are docs:
https://docs.getdbt.com/docs/get-started/getting-started/building-your-first-project/schedule-a-job#create-and-run-a-job

Finally, you'll need your dbt Cloud API key.
You can find it at the bottom of this page:
https://cloud.getdbt.com/next/settings/profile
"""

In [6]:
check_api_added_to_secrets = input("""Please add a secret to your dbt repo with the name DBTCLOUDAPIKEY
Instructions on how to do this below:
https://docs.github.com/en/codespaces/managing-codespaces-for-your-organization/managing-encrypted-secrets-for-your-repository-and-organization-for-github-codespaces
""")

Please add a secret to your dbt repo with the name DBTCLOUDAPIKEY
Instructions on how to do this below:
https://docs.github.com/en/codespaces/managing-codespaces-for-your-organization/managing-encrypted-secrets-for-your-repository-and-organization-for-github-codespaces



In [7]:
dbt_cloud_job_url = input("""Now please go to your cloud job in dbt Cloud, copy the URL, and paste it here.
It should look something like this: https://cloud.getdbt.com/next/deploy/1713/projects/150080/jobs/121379
""")

Now please go to your cloud job in dbt Cloud, copy the URL, and paste it here.
It should look something like this: https://cloud.getdbt.com/next/deploy/1713/projects/150080/jobs/121379
https://cloud.getdbt.com/next/deploy/1713/projects/150080/jobs/121379


In [8]:
dbt_cloud_api_run_url = re.sub("projects/d+/",
                               "",
                               dbt_cloud_job_url.replace("next/deploy/","api/v2/accounts/")
                              )+"/run/"

In [9]:
workflow_yml['jobs']['dbt_run']['steps'][0]['run'].replace('{dbt_cloud_api_run_url}',dbt_cloud_api_run_url)

'RESPONSE=$(curl -v -H "Authorization:Token ${{ secrets.DBTCLOUDAPIKEY }}" -H "Content-Type:application/json" -d \'{"cause":"Triggered by Github Actions","git_branch":"main"}\' https://cloud.getdbt.com/api/v2/accounts/1713/jobs/121379/run/)\nSTATUSCODE=$(echo "$RESPONSE" | jq \'.status.code\')\necho "$RESPONSE" | jq \'.status.user_message\'\nif [[ $STATUSCODE != 200 ]]; then echo "$RESPONSE" && bash -c "exit 1"; fi\n'

In [12]:
github_folder_path = os.path.join(dbt_base_path,".github")
if not os.path.exists(github_folder_path):
    os.mkdir(github_folder_path)
workflow_folder_path = os.path.join(github_folder_path,"workflows")
if not os.path.exists(workflow_folder_path):
    os.mkdir(workflow_folder_path)
workflow_path = os.path.join(workflow_folder_path,"dbt_run_on_merge.yml")

In [13]:
with open(workflow_path, "w") as f:
    yaml.dump(workflow_path, f)